# 🌿 Analyse des périmètres irrigués (Tunisie) — EDA + KPI + Visualisation

**Dataset :** `perimetres-irrigues-public.xlsx`  
**Objectif :** analyser l'évolution des superficies (irrigables, irriguées, exploitées) et calculer des KPI + taux.

> Notebook pensé pour **Google Colab** : upload le fichier Excel ou monte Google Drive.


In [ ]:
# 1) Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", 50)


## 2) Charger les données

In [ ]:
DATA_PATH = "../data/perimetres-irrigues-public.xlsx"
df_raw = pd.read_excel(DATA_PATH)

print("Shape:", df_raw.shape)
display(df_raw.head())
display(df_raw.dtypes)


## 3) Data processing

- Nettoyage des noms de colonnes
- Conversion numérique (si besoin)
- Création de KPI : taux d'irrigation et taux d'exploitation


In [ ]:
df = df_raw.copy()
df.columns = [c.strip() for c in df.columns]

# Harmoniser les noms (optionnel)
df = df.rename(columns={
    "Nom Délégation": "Nom_Delegation_fr",
    "Nom Delegation_arabe": "Nom_Delegation_ar",
    "Superficies irrigables_ha": "Irrigable_ha",
    "Superficies irriguées_ha": "Irriguee_ha",
    "Superficies exploitées_ha": "Exploitee_ha",
})

# Types
df["Année"] = pd.to_numeric(df["Année"], errors="coerce").astype("Int64")
for col in ["Irrigable_ha","Irriguee_ha","Exploitee_ha"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# Taux (en %)
df["Taux_irrigation_%"] = (df["Irriguee_ha"] / df["Irrigable_ha"]) * 100
df["Taux_exploitation_%"] = (df["Exploitee_ha"] / df["Irrigable_ha"]) * 100

df = df.drop_duplicates()
display(df.head())


## 4) Qualité des données

In [ ]:
na = (df.isna().mean()*100).round(2).sort_values(ascending=False)
display(na.to_frame("%NA"))

plt.figure(figsize=(8,3.5))
plt.bar(na.index.astype(str), na.values)
plt.xticks(rotation=45, ha="right")
plt.title("Valeurs manquantes (%)")
plt.tight_layout()
plt.show()


## 5) KPI Dashboard

In [ ]:
kpi = {
    "Nb lignes": len(df),
    "Nb délégations (FR)": df["Nom_Delegation_fr"].nunique(dropna=True),
    "Nb années": df["Année"].nunique(dropna=True),
    "Total irrigable (ha)": float(df["Irrigable_ha"].sum()),
    "Total irriguée (ha)": float(df["Irriguee_ha"].sum()),
    "Total exploitée (ha)": float(df["Exploitee_ha"].sum()),
    "Taux irrigation moyen (%)": float(df["Taux_irrigation_%"].mean()),
    "Taux exploitation moyen (%)": float(df["Taux_exploitation_%"].mean()),
}
kpi_df = pd.DataFrame({"KPI": kpi.keys(), "Valeur": kpi.values()})
display(kpi_df)


## 6) Visualisations

### 6.1 Évolution annuelle (sommes)

In [ ]:
year = df.groupby("Année")[["Irrigable_ha","Irriguee_ha","Exploitee_ha"]].sum().sort_index()

plt.figure(figsize=(8,4))
plt.plot(year.index.astype(int), year["Irrigable_ha"], marker="o", label="Irrigable")
plt.plot(year.index.astype(int), year["Irriguee_ha"], marker="o", label="Irriguée")
plt.plot(year.index.astype(int), year["Exploitee_ha"], marker="o", label="Exploitée")
plt.title("Évolution annuelle des superficies (ha)")
plt.xlabel("Année"); plt.ylabel("Hectares")
plt.legend()
plt.tight_layout()
plt.show()

display(year)


### 6.2 Taux moyens par année

In [ ]:
ty = df.groupby("Année")[["Taux_irrigation_%","Taux_exploitation_%"]].mean().sort_index()

plt.figure(figsize=(8,4))
plt.plot(ty.index.astype(int), ty["Taux_irrigation_%"], marker="o", label="Taux irrigation (%)")
plt.plot(ty.index.astype(int), ty["Taux_exploitation_%"], marker="o", label="Taux exploitation (%)")
plt.title("Évolution des taux moyens (%)")
plt.xlabel("Année"); plt.ylabel("%")
plt.legend()
plt.tight_layout()
plt.show()

display(ty)


### 6.3 Top délégations (taux irrigation moyen)

In [ ]:
top = (df.groupby("Nom_Delegation_fr")["Taux_irrigation_%"].mean()
       .sort_values(ascending=False).head(10))

plt.figure(figsize=(10,4))
plt.bar(top.index.astype(str), top.values)
plt.xticks(rotation=60, ha="right")
plt.title("Top 10 délégations — Taux d'irrigation moyen (%)")
plt.tight_layout()
plt.show()

display(top.to_frame("Taux_irrigation_moyen_%"))


### 6.4 Scatter : Irrigable vs Irriguée

In [ ]:
plt.figure(figsize=(6,4))
plt.scatter(df["Irrigable_ha"], df["Irriguee_ha"])
plt.title("Irrigable (ha) vs Irriguée (ha)")
plt.xlabel("Irrigable (ha)")
plt.ylabel("Irriguée (ha)")
plt.tight_layout()
plt.show()


### 6.5 Heatmap : délégation × année (taux irrigation)

In [ ]:
pivot = df.pivot_table(index="Nom_Delegation_fr", columns="Année", values="Taux_irrigation_%", aggfunc="mean")
# garder les 12 délégations les plus fréquentes (lisible)
top12 = df["Nom_Delegation_fr"].value_counts().head(12).index
p2 = pivot.loc[pivot.index.isin(top12)]

plt.figure(figsize=(9,4.8))
plt.imshow(p2.values, aspect="auto")
plt.colorbar(label="Taux irrigation (%)")
plt.xticks(range(len(p2.columns)), p2.columns.astype(int))
plt.yticks(range(len(p2.index)), p2.index.astype(str))
plt.title("Heatmap — Taux irrigation (%) | Top 12 délégations × années")
plt.tight_layout()
plt.show()

display(p2)


## 7) Exports (optionnel)

In [ ]:
df.to_csv("perimetres_irrigues_clean.csv", index=False, encoding="utf-8")
kpi_df.to_csv("kpi_perimetres_irrigues.csv", index=False, encoding="utf-8")
print("✅ Exports générés.")
